In [1]:
# Load libraries

# # Plotting utils 
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
import matplotlib.ticker as ticker 
import matplotlib.patches as patches
import matplotlib as matplotlib
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Analysis
import numpy as np 
import xarray as xr
import datetime
from   datetime import date, timedelta
import pandas as pd 
import scipy.stats as stats
## Need to use metPy conda env
import metpy.calc as mpc
from metpy.units import units
import pickle

# Import Ngl with pyn_env active 
import Ngl


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:

# Grabbed from Brian M. to use time midpoints, not end periods
def cesm_correct_time(ds):
    """Given a Dataset, check for time_bnds,
       and use avg(time_bnds) to replace the time coordinate.
       Purpose is to center the timestamp on the averaging inverval.   
       NOTE: ds should have been loaded using `decode_times=False`
    """
    assert 'time_bnds' in ds
    assert 'time' in ds
    correct_time_values = ds['time_bnds'].mean(dim='nbnd')
    # copy any metadata:
    correct_time_values.attrs = ds['time'].attrs
    ds = ds.assign_coords({"time": correct_time_values})
    ds = xr.decode_cf(ds)  # decode to datetime objects
    return ds


In [4]:
'''
Define a function to get the height of the PBL as the level with maximum d(var)/dz. 
Inputs:  A dataset with CAM output ('DS') and the variable to differentiate ('pbl_var')
Outputs: An array with boundary layer depth
'''
def PBLHasMaxDZ(DS, pbl_var): 
    # Convert HMGcamDS_all to height (nabbed from Rich's script)
    p0 = DS['P0'].values[0]
    
    plevm = DS['hyam']*p0 + DS['hybm']*DS['PS'].isel(lat=0,lon=0) # Mid level
    plevm.attrs['units'] = "Pa"

    # Height with standard atmosphere
    zlevm      = plevm
    zlevm_vals = 1000.*np.asarray(mpc.pressure_to_height_std(plevm)) # Units of [m] after multiplied 
    zlevm      = plevm.copy(deep=True)
    zlevm[:,:] = zlevm_vals
    
    pvar        = DS[pbl_var].isel(lat=0,lon=0)
    pvar['lev'] = zlevm[0,:].values
    dvardz      = pvar.differentiate("lev") # Find field gradient wrt HEIGHT!

    dvardz.loc[:,200:]   = 0.  # Restrict to a specificheight region
    dvardz.loc[:,:3000.] = 0

    nT = np.shape(dvardz)[0]
    PBLdepth = np.full([nT], np.nan)

    for iT in range(nT):
        iLevs  = np.where((zlevm[iT,:]>=200) & (zlevm[iT,:]<=3000))[0]
        maxLev = np.where(dvardz[iT,iLevs]==np.nanmax(dvardz[iT,iLevs]))[0]
        PBLdepth[iT] = zlevm[iT,iLevs[maxLev[0]]]
    
    return PBLdepth

In [5]:
def interpolateToPressure_v2(DS, varName, pressGoals):
#     nCases = len(DSin.case.values)
#     nTimes = len(DSin.time.values)
    
#     saveOut = np.full([nTimes,len(pressGoals),1,1], np.nan)

    ## For the larger arrays, need to operate case-by-case; input to vinth2p can only be 3 or 4 dimensions. 
#     for iCase in range(nCases): 
#     DS = DSin

    p0mb = DS.P0.values[0]/100        # mb

    # Pull out hya/hyb profiles 
    hyam = DS.hyam.values[0,:]
    hybm = DS.hybm.values[0,:]
    hyai = DS.hyai.values[0,:]
    hybi = DS.hybi.values[0,:]

    # Surface pressure with time dimension
    PS   = DS.PS.values              # Pa 

    # Converting variables: 
    if np.shape(DS[varName].values)[1]==len(DS.ilev.values):
        varInterp = Ngl.vinth2p(DS[varName].values,hyai,hybi,pressGoals,PS,1,p0mb,1,True)
    elif np.shape(DS[varName].values)[1]==len(DS.lev.values):
        varInterp = Ngl.vinth2p(DS[varName].values,hyam,hybm,pressGoals,PS,1,p0mb,1,True)

    saveOut = varInterp
    
    return saveOut

## Read in data

In [6]:
## Read in CLM files 

# This set uses theta, not theta_v; and adds outputs of rt'thv' and tau
# ## HTG * 10 case
# dataDir  = '/Users/mdfowler/Documents/Analysis/CLASP/SCAM_runs/FullyCoupledFromCTSM/FinalOption/betterFinidat4_MoreMomentsNoReorder/5minCoupling/realSfc_HTGtimes10/'
# caseName = 'FSCAM.T42_T42.CLASP_fullycoupled_FinalOption.onlyThlRt.HTG_betterFinidat4_MoreMomentsNoReorder_5mDt_realSfc10htg_useTheta_'


# ## HTG * 0.5 case
# dataDir  = '/Users/mdfowler/Documents/Analysis/CLASP/SCAM_runs/FullyCoupledFromCTSM/FinalOption/betterFinidat4_MoreMomentsNoReorder/5minCoupling/realSfc_HTGtimes0p5/'
# caseName = 'FSCAM.T42_T42.CLASP_fullycoupled_FinalOption.onlyThlRt.HTG_betterFinidat4_MoreMomentsNoReorder_5mDt_realSfc0p5htg_useTheta_'

## HTG * 0.0 case
dataDir  = '/Users/mdfowler/Documents/Analysis/CLASP/SCAM_runs/FullyCoupledFromCTSM/FinalOption/betterFinidat4_MoreMomentsNoReorder/5minCoupling/realSfc_HTGtimes0/'
caseName = 'FSCAM.T42_T42.CLASP_fullycoupled_FinalOption.onlyThlRt.HTG_betterFinidat4_MoreMomentsNoReorder_5mDt_realSfc0p0htg_useTheta_'


yearStrings  = np.asarray(['2015','2016','2017','2018'])
dateEndFiles = np.asarray([ '-05-31-84585.nc',
                            '-06-09-56985.nc',
                            '-06-18-29385.nc',
                            '-06-27-01785.nc',
                            '-07-05-60585.nc',
                            '-07-14-32985.nc',
                            '-07-23-05385.nc',
                            '-07-31-64185.nc',
                            '-08-09-36585.nc',
                            '-08-18-08985.nc',
                            '-08-26-67785.nc' ])

fileCount=0
for iYr in range(len(yearStrings)): 
    fileStart_atm_HTG  = dataDir+caseName+yearStrings[iYr]+'jja_try2.cam.h1.'+yearStrings[iYr]
    fileStart_HTG      = dataDir+caseName+yearStrings[iYr]+'jja_try2.clm2.h0.'+yearStrings[iYr]
    fileStartPatch_HTG = dataDir+caseName+yearStrings[iYr]+'jja_try2.clm2.h1.'+yearStrings[iYr]
        
    for iFile in range(len(dateEndFiles)):
        fileName_atm_HTG  = fileStart_atm_HTG+dateEndFiles[iFile]
        fileName_HTG      = fileStart_HTG+dateEndFiles[iFile]
        fileNamePatch_HTG = fileStartPatch_HTG+dateEndFiles[iFile]

        with xr.open_dataset(fileName_atm_HTG, decode_times=False) as HTG_camDS:
            HTG_camDS = cesm_correct_time(HTG_camDS)
            HTG_camDS['time'] = HTG_camDS.indexes['time'].to_datetimeindex()  
            
        with xr.open_dataset(fileName_HTG, decode_times=True) as HTG_clmDS: 
            HTG_clmDS['time'] = HTG_camDS['time']

        with xr.open_dataset(fileNamePatch_HTG, decode_times=True) as HTG_clmPatchDS: 
            HTG_clmPatchDS['time'] = HTG_camDS['time']
        
        # Discard the first two days if iFile == 0  
        if iFile==0:
            iTimeStart   = np.where(HTG_camDS.time.values >= (HTG_camDS.time.values[0] + np.timedelta64(2,'D')))[0]
            timeArr      = np.arange(iTimeStart[0], len(HTG_camDS.time.values))

            HTG_camDS      = HTG_camDS.isel(time=timeArr)
            HTG_clmDS      = HTG_clmDS.isel(time=timeArr)
            HTG_clmPatchDS = HTG_clmPatchDS.isel(time=timeArr)

        if fileCount==0:
            HTGcamDS_all = HTG_camDS
            HTGclmDS_realSfc = HTG_clmDS
            HTGclmDS_realSfcPatch = HTG_clmPatchDS
            
        else: 
            HTGcamDS_all = xr.concat([HTGcamDS_all,HTG_camDS], dim='time')
            HTGclmDS_realSfc = xr.concat([HTGclmDS_realSfc,HTG_clmDS], dim='time', data_vars='minimal')
            HTGclmDS_realSfcPatch = xr.concat([HTGclmDS_realSfcPatch,HTG_clmPatchDS], dim='time', data_vars='minimal')
        
        fileCount = fileCount+1
        print('Done with file %i of %i '% (fileCount,len(yearStrings)*len(dateEndFiles)))


Done with file 1 of 44 
Done with file 2 of 44 
Done with file 3 of 44 
Done with file 4 of 44 
Done with file 5 of 44 
Done with file 6 of 44 
Done with file 7 of 44 
Done with file 8 of 44 
Done with file 9 of 44 
Done with file 10 of 44 
Done with file 11 of 44 
Done with file 12 of 44 
Done with file 13 of 44 
Done with file 14 of 44 
Done with file 15 of 44 
Done with file 16 of 44 
Done with file 17 of 44 
Done with file 18 of 44 
Done with file 19 of 44 
Done with file 20 of 44 
Done with file 21 of 44 
Done with file 22 of 44 
Done with file 23 of 44 
Done with file 24 of 44 
Done with file 25 of 44 
Done with file 26 of 44 
Done with file 27 of 44 
Done with file 28 of 44 
Done with file 29 of 44 
Done with file 30 of 44 
Done with file 31 of 44 
Done with file 32 of 44 
Done with file 33 of 44 
Done with file 34 of 44 
Done with file 35 of 44 
Done with file 36 of 44 
Done with file 37 of 44 
Done with file 38 of 44 
Done with file 39 of 44 
Done with file 40 of 44 
Done with

In [7]:
## Add other vars to the list

# This set uses theta, not theta_v; and adds outputs of rt'thv' and tau
# ## HTG * 10 case
# dataDir  = '/Users/mdfowler/Documents/Analysis/CLASP/SCAM_runs/FullyCoupledFromCTSM/FinalOption/betterFinidat4_MoreMomentsNoReorder/5minCoupling/realSfc_HTGtimes10/'
# caseName = 'FSCAM.T42_T42.CLASP_fullycoupled_FinalOption.onlyThlRt.HTG_betterFinidat4_MoreMomentsNoReorder_5mDt_realSfc10htg_useTheta_'


# ## HTG * 0.5 case
# dataDir  = '/Users/mdfowler/Documents/Analysis/CLASP/SCAM_runs/FullyCoupledFromCTSM/FinalOption/betterFinidat4_MoreMomentsNoReorder/5minCoupling/realSfc_HTGtimes0p5/'
# caseName = 'FSCAM.T42_T42.CLASP_fullycoupled_FinalOption.onlyThlRt.HTG_betterFinidat4_MoreMomentsNoReorder_5mDt_realSfc0p5htg_useTheta_'

## HTG * 0.0 case
dataDir  = '/Users/mdfowler/Documents/Analysis/CLASP/SCAM_runs/FullyCoupledFromCTSM/FinalOption/betterFinidat4_MoreMomentsNoReorder/5minCoupling/realSfc_HTGtimes0/'
caseName = 'FSCAM.T42_T42.CLASP_fullycoupled_FinalOption.onlyThlRt.HTG_betterFinidat4_MoreMomentsNoReorder_5mDt_realSfc0p0htg_useTheta_'

yearStrings  = np.asarray(['2015','2016','2017','2018'])

fileCount=0
for iYr in range(len(yearStrings)): 
    
    fileStart_atm_HTG = dataDir+caseName+yearStrings[iYr]+'jja_try2.cam.h0.'
    fileName_atm_HTG  = fileStart_atm_HTG+'ExtraVarsAndBudget.nc'

    with xr.open_dataset(fileName_atm_HTG, decode_times=False) as HTG_camDS:
        HTG_camDS = cesm_correct_time(HTG_camDS)
        HTG_camDS['time'] = HTG_camDS.indexes['time'].to_datetimeindex()  
            
    # Discard first two days 
    iTimeStart   = np.where(HTG_camDS.time.values >= (HTG_camDS.time.values[0] + np.timedelta64(2,'D')))[0]
    timeArr      = np.arange(iTimeStart[0], len(HTG_camDS.time.values))

    HTG_camDS    = HTG_camDS.isel(time=timeArr)

    if fileCount==0:
        HTGcamDS_realSfcExtra = HTG_camDS
    else: 
        HTGcamDS_realSfcExtra = xr.concat([HTGcamDS_realSfcExtra,HTG_camDS], dim='time')
            
    fileCount = fileCount+1
    print('Done with file %i of %i '% (fileCount,len(yearStrings)))



Done with file 1 of 4 
Done with file 2 of 4 
Done with file 3 of 4 
Done with file 4 of 4 


In [8]:
# Merge into larger dataset
HTGcamDS_all = xr.merge([HTGcamDS_all, HTGcamDS_realSfcExtra])


## Now do the processing


In [9]:
## Add evaporative fraction to DS 
HTG_EF = HTGcamDS_all.LHFLX.values / (HTGcamDS_all.LHFLX.values + HTGcamDS_all.SHFLX.values)
HTGcamDS_all['EvapFraction'] = (('time'), np.squeeze(HTG_EF))

## Define the actual vertical velocity skew, not just the third order moment 
skw_W_HTG = HTGcamDS_all.WP3_CLUBB.values / ((HTGcamDS_all.WP2_CLUBB.values)**1.5)
HTGcamDS_all['Skw_W'] = (('time','ilev'), np.squeeze(skw_W_HTG))


## Steps to get PBL 

## Get *potential* temperature, not just T 

# So first, get actual pressures 
p0       = HTGcamDS_all['P0'].values[0]
plevmHTG = HTGcamDS_all['hyam']*p0 + HTGcamDS_all['hybm']*HTGcamDS_all['PS'].isel(lat=0,lon=0) # Mid level
plevmHTG.attrs['units'] = "Pa"

HTG_theta = np.asarray(mpc.potential_temperature(plevmHTG * units.pascals, HTGcamDS_all['T'] * units.kelvin))

# Add to existing DS
HTGcamDS_all['theta'] = (('time','lev','lat','lon'), HTG_theta)

# Height with standard atmosphere
zlevmHTG_vals = 1000.*np.asarray(mpc.pressure_to_height_std(plevmHTG)) # Units of [m] after multiplied 
zlevmHTG      = plevmHTG.copy(deep=True)
zlevmHTG[:,:] = zlevmHTG_vals

# Now compute the BL depth and save it to the larger CAM datasets 
PBLdepth_htg = PBLHasMaxDZ(HTGcamDS_all, 'theta')
print('Done computing PBL depth with theta')

# Add above to each dataset
HTGcamDS_all['PBLdepth']    = (('time'), PBLdepth_htg)



Done computing PBL depth with theta


**Convert to local time**

In [10]:
## Convert to local times...
HTGcamDS_localReal      = HTGcamDS_all.copy(deep=True)
HTGclmDS_localReal      = HTGclmDS_realSfc.copy(deep=True)
HTGclmDS_localRealPatch = HTGclmDS_realSfcPatch.copy(deep=True)


# Confirmed that all the times are identical, so using the same local time arrays
localTimes = HTGcamDS_all['time'].values - np.timedelta64(5,'h')

# Replace time dimension with local time
HTGcamDS_localReal      = HTGcamDS_localReal.assign_coords({"time": localTimes})
HTGclmDS_localReal      = HTGclmDS_localReal.assign_coords({"time": localTimes})
HTGclmDS_localRealPatch = HTGclmDS_localRealPatch.assign_coords({"time": localTimes})


print('First four times in UTC:\n', HTGcamDS_all.time.values[0:5])
print('First four times in local:\n', HTGcamDS_localReal.time.values[0:5])


First four times in UTC:
 ['2015-06-02T23:32:15.000000000' '2015-06-02T23:37:15.000000000'
 '2015-06-02T23:42:15.000000000' '2015-06-02T23:47:15.000000000'
 '2015-06-02T23:52:15.000000000']
First four times in local:
 ['2015-06-02T18:32:15.000000000' '2015-06-02T18:37:15.000000000'
 '2015-06-02T18:42:15.000000000' '2015-06-02T18:47:15.000000000'
 '2015-06-02T18:52:15.000000000']


In [11]:
## Save CLM files out to a pickle:
saveDir = '/Users/mdfowler/Documents/Analysis/CLASP/SCAM_runs/FullyCoupledFromCTSM/FinalOption/ProcessedFiles/'

pickle.dump( HTGclmDS_localReal,      open( saveDir+"HTGclmDS_localReal0p0.p", "wb" ) )
pickle.dump( HTGclmDS_localRealPatch, open( saveDir+"HTGclmDS_localRealPatch0p0.p", "wb" ) )



**More processing**

In [12]:
## Belated realization that the heights computed are above *sea level* not above ground level. 
#    Need to subtract elevation. 
nateFile = '/Users/mdfowler/Documents/Analysis/CLASP/NateForcing/clasp-hmg.bdate.nc'
nateDS = xr.open_dataset(nateFile, decode_times=True)
elevation = nateDS.alt.values

HTGcamDS_localReal['PBLdepth']   = HTGcamDS_localReal['PBLdepth']  - elevation[0][0] 


In [13]:
## Add in TKE 
HTGcamDS_localReal['TKE']  = (('time','ilev','lat,','lon'),
                   0.5*(HTGcamDS_localReal['UP2_CLUBB']+HTGcamDS_localReal['VP2_CLUBB']+HTGcamDS_localReal['WP2_CLUBB'])) 

HTGcamDS_localReal['TKE'].attrs['units']   = 'm2/s2'
HTGcamDS_localReal['TKE'].attrs['long_name']   = 'Turbulent Kinetic Energy'


In [14]:
## Add in wind speed 
HTGcamDS_localReal['WindMagnitude']  = (('time','lev','lat,','lon'),
                                np.sqrt((HTGcamDS_localReal.U.values**2.0) + (HTGcamDS_localReal.V.values**2.0)) )

HTGcamDS_localReal['WindMagnitude'].attrs['units']   = 'm/s'
HTGcamDS_localReal['WindMagnitude'].attrs['long_name']   = 'Wind speed'


**Interpolate to standard pressure levels**

In [15]:
## Decide on levels to interpoalte to and add to larger arrays
pnew64 = np.arange(200.0,980.0,10.0) 

HTGcamDS_localReal = HTGcamDS_localReal.assign_coords({"levInterp": pnew64})

varSels = np.asarray(['THLP2_CLUBB','RTP2_CLUBB','RTPTHLP_CLUBB','WPRTP_CLUBB','WPTHLP_CLUBB','WP2_CLUBB','UP2_CLUBB',
                      'VP2_CLUBB','TKE','U','V','T','Q','OMEGA','RVMTEND_CLUBB','STEND_CLUBB','CLOUD',
                      'UPWP_CLUBB','VPWP_CLUBB','WP2RTP_CLUBB','THETAL','QRL','QRS','DCQ','WindMagnitude'])


for iVar in range(len(varSels)): 
    varUnits = HTGcamDS_localReal[varSels[iVar]].units
    varName  = HTGcamDS_localReal[varSels[iVar]].long_name
    
    # Interpolate variables and add to larger arrays 
    interpVar_real = interpolateToPressure_v2(HTGcamDS_localReal,     varSels[iVar], pnew64)
    
    HTGcamDS_localReal[varSels[iVar]+'_interp']  = (('time','levInterp','lat','lon'), interpVar_real)
     
    ## Assign attibutes 
    HTGcamDS_localReal[varSels[iVar]+'_interp'].attrs['units']     = varUnits
    HTGcamDS_localReal[varSels[iVar]+'_interp'].attrs['long_name'] = varName

    
    print('Done with variable %i of %i' % (iVar, len(varSels)))
    

Done with variable 0 of 25
Done with variable 1 of 25
Done with variable 2 of 25
Done with variable 3 of 25
Done with variable 4 of 25
Done with variable 5 of 25
Done with variable 6 of 25
Done with variable 7 of 25
Done with variable 8 of 25
Done with variable 9 of 25
Done with variable 10 of 25
Done with variable 11 of 25
Done with variable 12 of 25
Done with variable 13 of 25
Done with variable 14 of 25
Done with variable 15 of 25
Done with variable 16 of 25
Done with variable 17 of 25
Done with variable 18 of 25
Done with variable 19 of 25
Done with variable 20 of 25
Done with variable 21 of 25
Done with variable 22 of 25
Done with variable 23 of 25
Done with variable 24 of 25


In [16]:
saveDir = '/Users/mdfowler/Documents/Analysis/CLASP/SCAM_runs/FullyCoupledFromCTSM/FinalOption/ProcessedFiles/'
pickle.dump( HTGcamDS_localReal,   open( saveDir+"realSfc_HTG0p0_withInterp.p", "wb" ) )
print('Saved interp CAM files to pickle')


Saved interp CAM files to pickle


## Test out?

In [16]:
# saveDir = '/Users/mdfowler/Documents/Analysis/CLASP/SCAM_runs/FullyCoupledFromCTSM/FinalOption/ProcessedFiles/'

# realSfc_allCases   = pickle.load( open( saveDir+"realSfc_allCases_withInterp.p", "rb" ) )

In [17]:
# HTGcamDS_localReal = HTGcamDS_localReal.assign_coords({"case": 'HTGp5'})


In [ ]:
# list1 = set(realSfc_allCases.keys())
# list2 = set(HTGcamDS_localReal.keys())
# sameVarsCAM = list(list1 & list2)

# realSfc_test = xr.concat([realSfc_allCases[sameVarsCAM],
#                           HTGcamDS_localReal[sameVarsCAM]], "case")
